In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#Data prep
shot_df = pd.read_csv("Hackathon_sv_shot_summary_2015-16.txt", sep=" ")

#have 165 occurences of null for closet defender id
shot_df = shot_df.replace({'(null)': -1})

#shot_df = shot_df.drop(shot_df.index[222316])
shot_df = shot_df[shot_df['SEASON'] != "GAME_ID"]

#convert columns to numbers
shot_df[["SEASON", "GAME_ID", "SV_GAME_ID", "TEAM_ID", "SV_TEAM_ID", "PERSON_ID", "SV_PLAYER_ID", "PERIOD", "GAME_CLOCK", "WALL_CLOCK", "DRIBBLES", "SHOT_DIST", "TOUCH_TIME", "CLOSE_DEF_PERSON_ID", "CLOSE_DEF_SV_PLAYER_ID", "CLOSE_DEF_DIST", "PT_VALUE", "PTS"]] = shot_df[["SEASON", "GAME_ID", "SV_GAME_ID", "TEAM_ID", "SV_TEAM_ID", "PERSON_ID", "SV_PLAYER_ID", "PERIOD", "GAME_CLOCK", "WALL_CLOCK", "DRIBBLES", "SHOT_DIST", "TOUCH_TIME", "CLOSE_DEF_PERSON_ID", "CLOSE_DEF_SV_PLAYER_ID", "CLOSE_DEF_DIST", "PT_VALUE", "PTS"]].apply(pd.to_numeric)

#convert missed/made to 0/1
shot_df['SHOT_RESULT'] = shot_df['SHOT_RESULT'].replace({'missed': 0})
shot_df['SHOT_RESULT'] = shot_df['SHOT_RESULT'].replace({'made': 1})

#classify shot distances
shot_df.loc[shot_df['PT_VALUE'] == 3, 'SHOT_TYPE'] = "Three Pointer"
shot_df.loc[(shot_df['SHOT_DIST'] >= 15) & (shot_df['PT_VALUE'] == 2), 'SHOT_TYPE'] = "Long Two-Pointer"
shot_df.loc[(shot_df['SHOT_DIST'] >= 5) & (shot_df['SHOT_DIST'] < 15), 'SHOT_TYPE'] = "Midrange"
shot_df.loc[shot_df['SHOT_DIST']< 5, 'SHOT_TYPE'] = "Layup"

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,2,3,5,6,8,9,11,12,13,14,15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
#drop everything but shot distance
shot_df = shot_df[['SHOT_RESULT', "SHOT_DIST", "DRIBBLES", "CLOSE_DEF_DIST", "SV_PLAYER_ID"]]

shot_df.head()

,SHOT_RESULT,SHOT_DIST,DRIBBLES,CLOSE_DEF_DIST,SV_PLAYER_ID
1,0,40.10,3,0.68,3741
2,0,2.43,16,2.90,5187
3,1,21.09,0,4.03,5359
4,1,4.60,13,0.97,5187
5,0,22.59,3,4.52,4895


In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(shot_df.drop('SHOT_RESULT',axis=1), 
                                                    shot_df['SHOT_RESULT'], test_size=0.3, 
                                                    random_state=101)


In [12]:
from sklearn.svm import SVC

In [13]:
model = SVC()
model.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
predictions = model.predict(X_test)

In [15]:
from sklearn.metrics import classification_report,confusion_matrix

In [16]:
print(confusion_matrix(y_test,predictions))

[[27305  9264]
 [17474 12653]]


In [17]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

          0       0.61      0.75      0.67     36569
          1       0.58      0.42      0.49     30127

avg / total       0.60      0.60      0.59     66696

